# Training Regression - Reaction

# Import packages

In [2]:
import pandas as pd
from lightning import pytorch as pl
from pathlib import Path

from chemprop import data, featurizers, models, nn

# Change data inputs here

In [3]:
chemprop_dir = Path.cwd().parent
# input_path = chemprop_dir / "tests" / "data" / "regression" / "rxn" / "rxn.csv"
input_path = "dataIC50_edited.csv"
num_workers = 0  # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'Smiles'
target_columns = ['pIC50']

## Load data

In [4]:
df_input = pd.read_csv(input_path)
df_input

,ID,Molecule ChEMBL ID,Molecular Weight,Smiles,Standard Type,Standard Relation,Standard Value,Standard Units,pIC50
0,1,CHEMBL197469,485.49,O=C1NC(CCCN2CC=C(c3ccccc3)CC2)=NC2CCOCC12,IC50,=',0.02,nM,"1,169,897"
1,2,CHEMBL4161087,474.51,Cc1ccc(N2CCN(C(=O)c3cc(Cc4n[nH]c(=O)c5ccccc45)...,IC50,<',0.1,nM,11
2,3,CHEMBL4453414,461.47,NC(=O)c1c(F)ccc2[nH]c(C3CCCNC3)nc12,IC50,=',0.1,nM,11
3,4,CHEMBL3649771,482.52,O=c1[nH]nc2c3c(cccc13)N=C(CNCc1cccc(C(F)(F)F)c...,IC50,=',0.19,nM,"107,212,464"
4,5,CHEMBL3764951,346.79,CC(=O)Nc1ccc(C2Nc3cc(F)cc4c(=O)[nH]nc(c34)C2c2...,IC50,=',0.2,nM,"1,069,897"
...,...,...,...,...,...,...,...,...,...
2449,2450,CHEMBL3937292,316.57,CC[C@@H]1C(=O)N(C)c2cnc(Nc3ccc(Cc4nc5c(C(N)=O)...,IC50,=',500000,nM,"4,301,029,996"
2450,2451,CHEMBL496937,142.16,O=C(c1cc(Cc2n[nH]c(=O)c3ccccc23)ccc1F)N1CCn2c(...,IC50,=',900000,nM,"4,045,757,491"
2451,2452,CHEMBL332444,136.15,NC(=O)c1ccc(N)cc1,IC50,=',1800000,nM,"3,744,727,495"
2452,2453,CHEMBL4521985,200.03,Cc1ccc(C2c3n[nH]c(=O)c4cccc(c34)NC2c2ccc(CN(C)...,IC50,=',2200000,nM,"3,657,577,319"


## Load smiles and targets

In [5]:
smis = df_input.loc[:, smiles_column].values
ys = df_input.loc[:, target_columns].values

smis[:5], ys[:5]

(array(['O=C1NC(CCCN2CC=C(c3ccccc3)CC2)=NC2CCOCC12',
        'Cc1ccc(N2CCN(C(=O)c3cc(Cc4n[nH]c(=O)c5ccccc45)cc(F)c3F)CC2)cc1',
        'NC(=O)c1c(F)ccc2[nH]c(C3CCCNC3)nc12',
        'O=c1[nH]nc2c3c(cccc13)N=C(CNCc1cccc(C(F)(F)F)c1)N2',
        'CC(=O)Nc1ccc(C2Nc3cc(F)cc4c(=O)[nH]nc(c34)C2c2ncnn2C)cc1'],
       dtype=object),
 array([['1,169,897'],
        ['11'],
        ['11'],
        ['107,212,464'],
        ['1,069,897']], dtype=object))

## Get datapoints

In [6]:
all_data = [data.ReactionDatapoint.from_smi(smi, y) for smi, y in zip(smis, ys)]

ValueError: not enough values to unpack (expected 3, got 1)

## Perform data splitting for training, validation, and testing

In [6]:
mols = [d.rct for d in all_data]  # Can either split by reactants (.rct) or products (.pdt)
train_indices, val_indices, test_indices = data.make_split_indices(mols, "random", (0.8, 0.1, 0.1))
train_data, val_data, test_data = data.split_data_by_indices(
    all_data, train_indices, val_indices, test_indices
)

# Defining the featurizer

Reactions can be featurized using the ```CondensedGraphOfReactionFeaturizer``` (also labeled ```CGRFeaturizer```).


Use ```_mode``` keyword to set the mode by which a reaction should be featurized into a ```MolGraph```.

Options are can be found with ```featurizers.RxnMode.keys```

In [7]:
for key in featurizers.RxnMode.keys():
    print(key)

REAC_PROD
REAC_PROD_BALANCE
REAC_DIFF
REAC_DIFF_BALANCE
PROD_DIFF
PROD_DIFF_BALANCE


In [8]:
featurizer = featurizers.CondensedGraphOfReactionFeaturizer(mode_="PROD_DIFF")

## Get ReactionDatasets

In [9]:
train_dset = data.ReactionDataset(train_data[0], featurizer)
scaler = train_dset.normalize_targets()

val_dset = data.ReactionDataset(val_data[0], featurizer)
val_dset.normalize_targets(scaler)
test_dset = data.ReactionDataset(test_data[0], featurizer)

## Get dataloaders

In [10]:
train_loader = data.build_dataloader(train_dset, num_workers=num_workers)
val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)
test_loader = data.build_dataloader(test_dset, num_workers=num_workers, shuffle=False)

# Change Message-Passing Neural Network (MPNN) inputs here

## Message passing

Message passing blocks must be given the shape of the featurizer's outputs.

Options are `mp = nn.BondMessagePassing()` or `mp = nn.AtomMessagePassing()`

In [11]:
fdims = featurizer.shape # the dimensions of the featurizer, given as (atom_dims, bond_dims).
mp = nn.BondMessagePassing(*fdims)

## Aggregation

In [12]:
print(nn.agg.AggregationRegistry)

ClassRegistry {
    'mean': <class 'chemprop.nn.agg.MeanAggregation'>,
    'sum': <class 'chemprop.nn.agg.SumAggregation'>,
    'norm': <class 'chemprop.nn.agg.NormAggregation'>
}


In [13]:
agg = nn.MeanAggregation()

## Feed-Forward Network (FFN)

In [14]:
print(nn.PredictorRegistry)

ClassRegistry {
    'regression': <class 'chemprop.nn.predictors.RegressionFFN'>,
    'regression-mve': <class 'chemprop.nn.predictors.MveFFN'>,
    'regression-evidential': <class 'chemprop.nn.predictors.EvidentialFFN'>,
    'regression-quantile': <class 'chemprop.nn.predictors.QuantileFFN'>,
    'classification': <class 'chemprop.nn.predictors.BinaryClassificationFFN'>,
    'classification-dirichlet': <class 'chemprop.nn.predictors.BinaryDirichletFFN'>,
    'multiclass': <class 'chemprop.nn.predictors.MulticlassClassificationFFN'>,
    'multiclass-dirichlet': <class 'chemprop.nn.predictors.MulticlassDirichletFFN'>,
    'spectral': <class 'chemprop.nn.predictors.SpectralFFN'>
}


In [15]:
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)

In [16]:
ffn = nn.RegressionFFN(output_transform=output_transform)

## Batch norm

In [17]:
batch_norm = True

## Metrics

In [18]:
print(nn.metrics.MetricRegistry)

ClassRegistry {
    'mse': <class 'chemprop.nn.metrics.MSE'>,
    'mae': <class 'chemprop.nn.metrics.MAE'>,
    'rmse': <class 'chemprop.nn.metrics.RMSE'>,
    'bounded-mse': <class 'chemprop.nn.metrics.BoundedMSE'>,
    'bounded-mae': <class 'chemprop.nn.metrics.BoundedMAE'>,
    'bounded-rmse': <class 'chemprop.nn.metrics.BoundedRMSE'>,
    'r2': <class 'chemprop.nn.metrics.R2Score'>,
    'binary-mcc': <class 'chemprop.nn.metrics.BinaryMCCMetric'>,
    'multiclass-mcc': <class 'chemprop.nn.metrics.MulticlassMCCMetric'>,
    'roc': <class 'chemprop.nn.metrics.BinaryAUROC'>,
    'prc': <class 'chemprop.nn.metrics.BinaryAUPRC'>,
    'accuracy': <class 'chemprop.nn.metrics.BinaryAccuracy'>,
    'f1': <class 'chemprop.nn.metrics.BinaryF1Score'>
}


In [ ]:
metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()] 
# Only the first metric is used for training and early stopping

## Construct MPNN

In [20]:
mpnn = models.MPNN(mp, agg, ffn, batch_norm, metric_list)
mpnn

MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=134, out_features=300, bias=False)
    (W_h): Linear(in_features=300, out_features=300, bias=False)
    (W_o): Linear(in_features=406, out_features=300, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): MeanAggregation()
  (bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=300, out_features=300, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=300, out_features=1, bias=True)
      )
    )
    (criterion): MSE(task_weights=[[1.0]])
    (output_transform): UnscaleTransform()
  )
  (X_d_transform): Identity()
  (metrics): ModuleList(
    (0): RMSE(task_weights=[[1.0]])
    (1): MAE(task_weig

# Training and testing

## Set up trainer

In [21]:
trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True,  # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=20,  # number of epochs to train for
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


## Start training

In [22]:
trainer.fit(mpnn, train_loader, val_loader)

/home/knathan/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/knathan/chemprop/examples/checkpoints exists and is not empty.
Loading `train_dataloader` to estimate number of stepping batches.
/home/knathan/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 252 K  | train
1 | agg             | MeanAggregation    | 0      | train
2 | bn              | BatchNorm1d        | 600    | train
3 | predictor       | RegressionFFN      | 90.6 K | train
4 | X_d_transform   | Identity         

Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

/home/knathan/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 0:   0%|          | 0/2 [00:00<?, ?it/s]                             

Epoch 19: 100%|██████████| 2/2 [00:00<00:00,  7.13it/s, train_loss_step=0.0888, val_loss=0.937, train_loss_epoch=0.0274]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 2/2 [00:00<00:00,  6.38it/s, train_loss_step=0.0888, val_loss=0.937, train_loss_epoch=0.0274]


## Test results

In [23]:
results = trainer.test(mpnn, test_loader)

/home/knathan/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 59.59it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/mae          │     1.111189842224121     │
│         test/rmse         │    1.4387098550796509     │
└───────────────────────────┴───────────────────────────┘